<a href="https://colab.research.google.com/github/Mmarlo5/psychic-doodle/blob/main/Problem_Set_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I decided to change my research focus a little bit. I have decided that I will be looking to see if public opinion on abortion matches abortion restrictions across the United States. 
The first dataset comes from Pew Research Center and contains data on the views about Abortion by State (https://www.pewresearch.org/religion/religious-landscape-study/compare/views-about-abortion/by/state/) 

The second dataset comes from the State Policy Database and also contains data on the type of restrictions that each state has, so the common variable for all the datasets is "states." https://www.statepolicyindex.com/data/" *the data I used is contained in the CSV file titled "abortion."

The third dataset comes from (https://www.guttmacher.org/article/2022/03/2022-state-legislative-sessions-abortion-bans-and-restrictions-medication-abortion) which also contains more abservations on support of abortion by state which adds more observations to the first dataset. 

I chose those datasets because one provided the information I needed to determine public support of abortion, and the other dataset provided very detailed information on the type of restrictions in each state. The thid provided me with more observations for public opinion on abortion. 

My research question is: Do abortion restrictions in states, match public opinion on abortion in that state? 

Hypothesis: I hypothesis that public opinion and abortion restrictions are similar. 

**bold text**
Notes for me: 
Fix names so they match 
drop all variables that I do not need
Finally merge data together


In [ ]:

import time, os, sys, re 
import zipfile, json, datetime, string    
import numpy as np
from statistics import *

import matplotlib.pyplot as plt

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler

import urllib

from google.colab import files

import webbrowser                                                                      
import folium as f                                                                     
from folium.plugins import MarkerCluster, HeatMap                                      


! pip install geopandas 
import geopandas as gpd  

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 4.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [ ]:
abortion1 = pd.read_csv("https://raw.githubusercontent.com/Mmarlo5/psychic-doodle/main/Abortion%20support%20by%20state.csv")
abortion1.head()

,State,Legal in all/most cases,Illegal in all/most cases,Don't know,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Alabama,37%,58%,4%,NaN,NaN,NaN
1,Alaska,63%,34%,3%,NaN,NaN,NaN
2,Arizona,49%,46%,4%,NaN,NaN,NaN
3,Arkansas,38%,60%,2%,NaN,NaN,NaN
4,California,57%,38%,5%,NaN,NaN,NaN


In [ ]:
old_variable_Legal_in_all_most_cases = 42
new_variable_Mostly_Legal = old_variable_Legal_in_all_most_cases
print(new_variable_Mostly_Legal) # Output: 42
print(abortion1)

In [ ]:
abortion1.rename(columns={'old_Don&#39;t know': 'new_Do not know'}, inplace=True)
print(abortion1)
#had to drop each variable separately 
#Have to rename Legal in all/most cases and Illegal in all/most cases to reflect Abortion3 naming

In [ ]:
abortion1 = abortion1.drop(columns=['Unnamed: 5'])
print(abortion1)

In [ ]:
abortion1 = abortion1.drop(columns=['Unnamed: 6'])
print(abortion1)

In [ ]:
abortion1.head()
#why is "dont know" still coming up like that? 

In [111]:
abortion2 = pd.read_csv("https://raw.githubusercontent.com/Mmarlo5/psychic-doodle/main/Abortion%20Data2.csv")
abortion2.head()

,State,Pgestlim,Pparbth,Pwait,Unnamed: 4
0,Alabama,1,1,0,NaN
1,Alaska,0,0,0,NaN
2,Arizona,1,0,0,NaN
3,Arkansas,1,0,0,NaN
4,California,1,0,0,NaN


In [ ]:
abortion2 = abortion2.drop(columns=['Unnamed: 4'])
print(abortion1)

In [ ]:
abortion2.head()

In [ ]:
abortion3=pd.read_csv("https://raw.githubusercontent.com/Mmarlo5/psychic-doodle/main/Abortion%20data_3.csv")
abortion3.head()

In [114]:
mergedOuter = pd.merge(abortion1, abortion2, how='outer', on=['State'],indicator=True) #outer=keep all
mergedOuter

,State,Legal in all/most cases,Illegal in all/most cases,Don't know,Pgestlim,Pparbth,Pwait,Unnamed: 4,_merge
0,Alabama,37%,58%,4%,1.0,1.0,0.0,NaN,both
1,Alaska,63%,34%,3%,0.0,0.0,0.0,NaN,both
2,Arizona,49%,46%,4%,1.0,0.0,0.0,NaN,both
3,Arkansas,38%,60%,2%,1.0,0.0,0.0,NaN,both
4,California,57%,38%,5%,1.0,0.0,0.0,NaN,both
5,Colorado,59%,36%,5%,0.0,0.0,0.0,NaN,both
6,Connecticut,67%,28%,5%,1.0,0.0,0.0,NaN,both
7,Delaware,55%,38%,6%,1.0,0.0,0.0,NaN,both
8,District of Columbia,70%,26%,4%,NaN,NaN,NaN,NaN,left_only
9,Florida,56%,39%,5%,1.0,0.0,0.0,NaN,both
